In [1]:
!apt-get -q -y install sudo file mecab libmecab-dev mecab-ipadic-utf8 git curl python-mecab > /dev/null
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git > /dev/null 
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n > /dev/null 2>&1
!pip install mecab-python3 > /dev/null

# シンボリックリンクによるエラー回避
!ln -s /etc/mecabrc /usr/local/etc/mecabrc

Cloning into 'mecab-ipadic-neologd'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 75 (delta 5), reused 54 (delta 0), pack-reused 0
Unpacking objects: 100% (75/75), done.


In [2]:
# パスの確認↓
!echo `mecab-config --dicdir`"/mecab-ipadic-neologd"

/usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd


In [23]:
# 実行前にMeCabのインストールを必ず！

import MeCab
import os,json,random

dict_file = "markov_dict_ext.json"
dic = {}

text = "rsplitは、sep を区切り文字とした、文字列中の単語のリストを返します。 maxsplit が与えられた場合、文字列の 右端 から最大 maxsplit 回分割を行います。sep が指定されていない、あるいは None のとき、全ての空白文字が区切り文字となります。右から分割していくことを除けば、 rsplit() は後ほど詳しく述べる split() と同様に振る舞います。splitは、文字列を sep をデリミタ文字列として区切った単語のリストを返します。maxsplit が与えられていれば、最大で maxsplit 回分割されます。つまり、リストは最大 maxsplit+1 要素になります。maxsplit が与えられないか -1 なら、分割の回数に制限はありません。可能なだけ分割されます。sep が与えられた場合、連続した区切り文字はまとめられず、空の文字列を区切っていると判断されます。例えば '1,,2'.split(',') は ['1', '', '2'] を返します。引数 sep は複数の文字にもできます。例えば '1<>2<>3'.split('<>') は ['1', '2', '3'] を返します。区切り文字を指定して空の文字列を分割すると、 [''] を返します。"
text = text.split('。')
text.pop(-1)
text.append('さようなら')

# 辞書への登録
def regist_dic(wordlist):
    global dic
    w1 = ""
    w2 = ""
    
    # 要素が3未満の場合は、何もしない
    if len(wordlist) < 3 : return
    
    for w in wordlist :
        word = w[0]
        # wordlistに入っているものを出力してみる
        # print('wordlist{}:w[0]{}'.forma(wordlist,w[0]))
        if word == "" or  word == "\r\n" or word == "\n" : continue
        # 辞書に単語を設定
        # w1かw2が空白なら以下「set_dic」を処理しない
        if w1 and w2 :
            # print('w1{}, w2'.format(w1, w2))
            set_dic(dic, w1, w2, word)
        # 文末を表す語の場合、連鎖をクリアする
        if word == "。" or word == "?" or  word == "？" :
            w1 = ""
            w2 = ""
            continue
        # 次の前後関係を登録するために、単語をスライド
        w1, w2 = w2, word
    
    # 辞書を保存
    json.dump(dic, open(dict_file,"w", encoding="utf-8"))

# 辞書に単語を設定
def set_dic(dic, w1, w2, w3):
    # 新しい単語の場合は、新しい辞書オブジェクトを作成
    if w1 not in dic : dic[w1] = {}
    if w2 not in dic[w1] : dic[w1][w2] = {}
    if w3 not in dic[w1][w2]: dic[w1][w2][w3] = 0
    # 単語の出現数をインクリメントする
    dic[w1][w2][w3] += 1
# 応答文の作成
def make_response(word):
    res = []
    
    # 「名詞」/「形容詞」/「動詞」は、文章の意図を示していることが多いと想定し、始点の単語とする。
    w1 = word
    res.append(w1)
    w2 = word_choice(dic[w1])
    res.append(w2)
    while True:
        # w1,w2の組み合わせから予想される、単語を選択
        if w1 in dic and w2 in dic[w1] : w3 = word_choice(dic[w1][w2])
        else : w3 = ""
        res.append(w3)
        # 文末を表す語の場合、作文を終了
        if w3 == "。" or w3 == "?" or  w3 == "？"  or w3 == "" :  break
        # 次の単語を選択するために、単語をスライド
        w1, w2 = w2, w3
    return "".join(res)
        
def word_choice(candidate):
    keys = candidate.keys()
    return random.choice(list(keys))

# ====================================
#         メイン処理
# ====================================

# 辞書がすでに存在する場合は、最初に読み込む
if os.path.exists(dict_file):
        dic = json.load(open(dict_file,"r"))
        
for i in text:
    # 標準入力から入力を受け付け、「さようなら」が入力されるまで続ける
    print("You -> ", end='')
    print(i)
    text = i
    if text == "" or text == "さようなら" : 
        print("Bot -> さようなら")
        break

    # 文章整形（次のセルに解説あり）
    if text[-1] != "。" and text[-1] != "?" and text[-1] != "？" : text +="。"
    
    # 形態素解析
    # tagger = MeCab.Tagger("-d /var/lib/mecab/dic/mecab-ipadic-neologd")

    # 「パスの確認」のセルで表示されたパスをTaggerの引数に入れること
    # ※※※※※効果測定で使います！※※※※※※※
    tagger = MeCab.Tagger('/usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd')
    tagger.parse("") 
    node =  tagger.parseToNode(text)
    
    # 形態素解析の結果から、単語と品詞情報を抽出
    wordlist = []
    while node is not None:
        # node.surface：形態素解析された単語が入っている
        # node.feature：形態素の解析結果のパラメータで、「名詞,一般,*,*,*,*,月,ツキ,ツキ」のようなデータ
        # そのため、node.feature.split(',')[0]とすると、品詞（上記例だと「名詞」という文字列）が取得できる
        hinshi = node.feature.split(",")[0]

        # BOS:Beginning Of Sentence(文頭)、EOS:End Of Sentence(文末)
        if  hinshi not  in ["BOS/EOS"]:
            # wordlistにnode.surfaceとhinshiを追加
            wordlist.append([node.surface,hinshi])
            # print(wordlist)
        node = node.next
    
    # マルコフ連鎖の辞書に登録
    # print('regist_dicに登録するwordlist{}'.format(wordlist))
    regist_dic(wordlist)

    # 応答文の作成
    for w in wordlist:
        word = w[0]
        hinshi = w[1]
        # 品詞が「感動詞」の場合は、単語をそのまま返す
        if hinshi in [ "感動詞"] : 
            print("Bot -> " + word)
            break
        # 品詞が「名詞」「形容詞」「動詞」の場合で、かつ、辞書に単語が存在する場合は、作文して返す
        elif (hinshi in [ "名詞" ,"形容詞","動詞"]) and (word in dic):
            print("Bot -> " + make_response(word))
            break

You -> rsplitは、sep を区切り文字とした、文字列中の単語のリストを返します
Bot -> rsplit()は後ほど詳しく述べるsplit()と同様に振る舞います。
You ->  maxsplit が与えられた場合、文字列の 右端 から最大 maxsplit 回分割を行います
Bot -> maxsplit回分割されます。
You -> sep が指定されていない、あるいは None のとき、全ての空白文字が区切り文字となります
Bot -> sepをデリミタ文字列を分割すると、['']を返します。
You -> 右から分割していくことを除けば、 rsplit() は後ほど詳しく述べる split() と同様に振る舞います
Bot -> 右から分割していくことを除けば、rsplit()は後ほど詳しく述べるsplit()と同様に振る舞います。
You -> splitは、文字列を sep をデリミタ文字列として区切った単語のリストを返します
Bot -> split('<>')は['1','2']を返します。
You -> maxsplit が与えられていれば、最大で maxsplit 回分割されます
Bot -> maxsplit回分割を行います。
You -> つまり、リストは最大 maxsplit+1 要素になります
Bot -> リストは最大maxsplit回分割されます。
You -> maxsplit が与えられないか -1 なら、分割の回数に制限はありません
Bot -> maxsplitが与えられた場合、連続した区切り文字はまとめられず、空の文字列として区切った単語のリストを返します。
You -> 可能なだけ分割されます
Bot -> 可能なだけ分割されていない、あるいはNoneのとき、全ての空白文字が区切り文字を指定していくことを除けば、rsplit()は後ほど詳しく述べるsplit()と同様に振る舞います。
You -> sep が与えられた場合、連続した区切り文字はまとめられず、空の文字列を区切っていると判断されます
Bot -> sepを区切り文字となります。
You -> 例えば '1,,2'.split(',') は ['1', '', '2'] を返します
Bot -> '3']を返します。
You -> 引数 sep 